In [1]:
! mkdir _data

mkdir: cannot create directory ‘_data’: File exists


In [2]:
! wget -P _data https://archive.ics.uci.edu/ml/machine-learning-databases/diabetes/diabetes-data.tar.Z

--2019-08-28 17:14:39--  https://archive.ics.uci.edu/ml/machine-learning-databases/diabetes/diabetes-data.tar.Z
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 187551 (183K) [application/x-httpd-php]
Saving to: ‘_data/diabetes-data.tar.Z.1’

diabetes-data.tar.Z 100%[===================>] 183.16K   908KB/s    in 0.2s    

2019-08-28 17:14:40 (908 KB/s) - ‘_data/diabetes-data.tar.Z.1’ saved [187551/187551]



In [3]:
! tar xzf _data/diabetes-data.tar.Z -C _data

In [4]:
# Imports 
import glob
import pandas as pd
import datetime
import time
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [5]:
all_files = sorted(glob.glob('_data/Diabetes-Data/data*'))
df_list = []
for idx,filename in enumerate(all_files):
    df = pd.read_csv(filename, sep='\t', header=None)
    df['patient_id'] = idx+1
    df_list.append(df)
df = pd.concat(df_list, axis=0, ignore_index=True)
df.shape

(29330, 5)

In [6]:
# Drop nulls
df = df.dropna()
df

,0,1,2,3,patient_id
0,04-21-1991,9:09,58,100,1
1,04-21-1991,9:09,33,9,1
2,04-21-1991,9:09,34,13,1
3,04-21-1991,17:08,62,119,1
4,04-21-1991,17:08,33,7,1
...,...,...,...,...,...
29325,05-09-1989,08:00,33,1,70
29326,05-09-1989,08:00,34,7,70
29327,05-10-1989,08:00,34,7,70
29328,05-11-1989,08:00,34,7,70


In [7]:
df[2].value_counts()

33    9485
34    3830
58    3518
62    3160
60    2771
48    1883
35    1053
57     990
64     904
65     331
67     326
63     219
66     154
70     139
56     119
71      98
72      94
69      68
61      66
68      34
59      20
36       1
4        1
Name: 2, dtype: int64

In [8]:
codes = """
33 = Regular insulin dose
34 = NPH insulin dose
35 = UltraLente insulin dose
48 = Unspecified blood glucose measurement
57 = Unspecified blood glucose measurement
58 = Pre-breakfast blood glucose measurement
59 = Post-breakfast blood glucose measurement
60 = Pre-lunch blood glucose measurement
61 = Post-lunch blood glucose measurement
62 = Pre-supper blood glucose measurement
63 = Post-supper blood glucose measurement
64 = Pre-snack blood glucose measurement
65 = Hypoglycemic symptoms
66 = Typical meal ingestion
67 = More-than-usual meal ingestion
68 = Less-than-usual meal ingestion
69 = Typical exercise activity
70 = More-than-usual exercise activity
71 = Less-than-usual exercise activity
72 = Unspecified special event
"""

In [9]:
code_dict = {}
for code in codes.split('\n')[1:-1]:
    key,val = code.split(' = ')
    code_dict[int(key)] = val
code_dict

{33: 'Regular insulin dose',
 34: 'NPH insulin dose',
 35: 'UltraLente insulin dose',
 48: 'Unspecified blood glucose measurement',
 57: 'Unspecified blood glucose measurement',
 58: 'Pre-breakfast blood glucose measurement',
 59: 'Post-breakfast blood glucose measurement',
 60: 'Pre-lunch blood glucose measurement',
 61: 'Post-lunch blood glucose measurement',
 62: 'Pre-supper blood glucose measurement',
 63: 'Post-supper blood glucose measurement',
 64: 'Pre-snack blood glucose measurement',
 65: 'Hypoglycemic symptoms',
 66: 'Typical meal ingestion',
 67: 'More-than-usual meal ingestion',
 68: 'Less-than-usual meal ingestion',
 69: 'Typical exercise activity',
 70: 'More-than-usual exercise activity',
 71: 'Less-than-usual exercise activity',
 72: 'Unspecified special event'}

In [10]:
# figure out missing codes
non_standard_codes = set(code_dict.keys()).symmetric_difference(set(df[2].unique()))
non_standard_codes

{4, 36, 56}

In [11]:
df[df[2].isin(non_standard_codes)]

,0,1,2,3,patient_id
1707,07-21-1990,08:56,56,115,3
1712,07-22-1990,05:56,56,108,3
1722,07-23-1990,05:04,56,110,3
1731,07-24-1990,05:27,56,116,3
1740,07-25-1990,06:24,56,121,3
...,...,...,...,...,...
23926,07-11-1990,21:15,56,52,57
23934,07-13-1990,21:53,56,162,57
23935,07-13-1990,21:56,56,91,57
28944,08-24-1990,13:02,56,258,69


In [12]:
# remove non standard codes
df = df.drop(df[df[2].isin(non_standard_codes)].index)

In [13]:
# standardize duplicate codes
df[2] = df[2].replace(48, 57)

In [14]:
# Some Dates are wrong
for d in df[0]:
    try:
        datetime.datetime.strptime(d, '%m-%d-%Y')
    except:
        print(d)

06-31-1991
06-31-1991
06-31-1991
06-31-1991
06-31-1991
06-31-1991
06-31-1991


In [15]:
# Some time are wrong
for t in df[1]:
    try:
        datetime.datetime.strptime(t, '%H:%M')
        
    except:
        print(t)

56:35
56:35
56:35
188:00
188:00


In [16]:
# check records
df[df[0] =='06-31-1991']

,0,1,2,3,patient_id
6505,06-31-1991,07:50,58,149,20
6506,06-31-1991,07:50,33,4,20
6507,06-31-1991,07:50,34,24,20
6508,06-31-1991,13:30,60,162,20
6509,06-31-1991,13:30,33,5,20
6510,06-31-1991,19:45,62,213,20
6511,06-31-1991,19:45,33,11,20


In [17]:
df[df[1].isin(['56:35', '188:00'])]

,0,1,2,3,patient_id
28070,04-09-1991,56:35,58,237,67
28071,04-09-1991,56:35,33,16,67
28072,04-09-1991,56:35,34,40,67
28171,04-21-1991,188:00,62,128,67
28172,04-21-1991,188:00,33,14,67


In [18]:
# other patient data looks normal
df[df['patient_id'] == 20]

,0,1,2,3,patient_id
6142,05-12-1991,06:55,58,223,20
6143,05-12-1991,06:55,33,6,20
6144,05-12-1991,06:55,34,24,20
6145,05-12-1991,11:20,60,260,20
6146,05-12-1991,11:20,33,8,20
...,...,...,...,...,...
7140,09-23-1991,12:15,33,5,20
7141,09-23-1991,17:15,62,264,20
7142,09-23-1991,17:15,33,13,20
7143,09-23-1991,21:10,57,256,20


In [19]:
df[df['patient_id'] == 67]

,0,1,2,3,patient_id
27278,01-01-1991,9:10,58,235,67
27279,01-01-1991,9:10,33,18,67
27280,01-01-1991,9:10,34,40,67
27281,01-01-1991,13:40,60,195,67
27282,01-01-1991,13:40,33,10,67
...,...,...,...,...,...
28240,04-30-1991,12:05,60,64,67
28241,04-30-1991,12:05,33,10,67
28242,04-30-1991,18:10,62,100,67
28243,04-30-1991,18:10,33,14,67


In [20]:
# safe to drop
df = df.drop(df[df[0] =='06-31-1991'].index)
df = df.drop(df[df[1].isin(['56:35', '188:00'])].index)


In [21]:
#Drop more invalid data
non_ints = set([])
for val in df[3]:
    try:
        int(val)
    except:
        non_ints.add(val)
        print(val)
df = df.drop(df[df[3].isin(non_ints)].index)


0Hi
0Hi
0Hi
0Hi
0Hi
0Lo
0Hi
0''


In [22]:
df['timestamp'] = df.apply(lambda x: datetime.datetime.strptime(f'{x[0]} {x[1]}', '%m-%d-%Y %H:%M'), axis=1)
df

,0,1,2,3,patient_id,timestamp
0,04-21-1991,9:09,58,100,1,1991-04-21 09:09:00
1,04-21-1991,9:09,33,9,1,1991-04-21 09:09:00
2,04-21-1991,9:09,34,13,1,1991-04-21 09:09:00
3,04-21-1991,17:08,62,119,1,1991-04-21 17:08:00
4,04-21-1991,17:08,33,7,1,1991-04-21 17:08:00
...,...,...,...,...,...,...
29325,05-09-1989,08:00,33,1,70,1989-05-09 08:00:00
29326,05-09-1989,08:00,34,7,70,1989-05-09 08:00:00
29327,05-10-1989,08:00,34,7,70,1989-05-10 08:00:00
29328,05-11-1989,08:00,34,7,70,1989-05-11 08:00:00


In [23]:
df[3] = df[3].astype(int)

In [24]:
df

,0,1,2,3,patient_id,timestamp
0,04-21-1991,9:09,58,100,1,1991-04-21 09:09:00
1,04-21-1991,9:09,33,9,1,1991-04-21 09:09:00
2,04-21-1991,9:09,34,13,1,1991-04-21 09:09:00
3,04-21-1991,17:08,62,119,1,1991-04-21 17:08:00
4,04-21-1991,17:08,33,7,1,1991-04-21 17:08:00
...,...,...,...,...,...,...
29325,05-09-1989,08:00,33,1,70,1989-05-09 08:00:00
29326,05-09-1989,08:00,34,7,70,1989-05-09 08:00:00
29327,05-10-1989,08:00,34,7,70,1989-05-10 08:00:00
29328,05-11-1989,08:00,34,7,70,1989-05-11 08:00:00


In [25]:
patient_1 =  df[df['patient_id']==1]
patient_1 = patient_1.pivot_table(index='timestamp', columns=2, values=3)
patient_1

2,33,34,57,58,60,62,65
timestamp,,,,,,,
1991-04-21 09:09:00,9.0,13.0,NaN,100.0,NaN,NaN,NaN
1991-04-21 17:08:00,7.0,NaN,NaN,NaN,NaN,119.0,NaN
1991-04-21 22:51:00,NaN,NaN,123.0,NaN,NaN,NaN,NaN
1991-04-22 07:35:00,10.0,13.0,NaN,216.0,NaN,NaN,NaN
1991-04-22 13:40:00,2.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1991-09-02 08:51:00,9.0,16.0,NaN,168.0,NaN,NaN,NaN
1991-09-02 13:00:00,4.0,NaN,NaN,NaN,NaN,NaN,NaN
1991-09-02 17:30:00,7.0,NaN,NaN,NaN,NaN,61.0,NaN


In [26]:
patient_1.rename(columns=code_dict)

2,Regular insulin dose,NPH insulin dose,Unspecified blood glucose measurement,Pre-breakfast blood glucose measurement,Pre-lunch blood glucose measurement,Pre-supper blood glucose measurement,Hypoglycemic symptoms
timestamp,,,,,,,
1991-04-21 09:09:00,9.0,13.0,NaN,100.0,NaN,NaN,NaN
1991-04-21 17:08:00,7.0,NaN,NaN,NaN,NaN,119.0,NaN
1991-04-21 22:51:00,NaN,NaN,123.0,NaN,NaN,NaN,NaN
1991-04-22 07:35:00,10.0,13.0,NaN,216.0,NaN,NaN,NaN
1991-04-22 13:40:00,2.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1991-09-02 08:51:00,9.0,16.0,NaN,168.0,NaN,NaN,NaN
1991-09-02 13:00:00,4.0,NaN,NaN,NaN,NaN,NaN,NaN
1991-09-02 17:30:00,7.0,NaN,NaN,NaN,NaN,61.0,NaN


In [27]:
code_dict

{33: 'Regular insulin dose',
 34: 'NPH insulin dose',
 35: 'UltraLente insulin dose',
 48: 'Unspecified blood glucose measurement',
 57: 'Unspecified blood glucose measurement',
 58: 'Pre-breakfast blood glucose measurement',
 59: 'Post-breakfast blood glucose measurement',
 60: 'Pre-lunch blood glucose measurement',
 61: 'Post-lunch blood glucose measurement',
 62: 'Pre-supper blood glucose measurement',
 63: 'Post-supper blood glucose measurement',
 64: 'Pre-snack blood glucose measurement',
 65: 'Hypoglycemic symptoms',
 66: 'Typical meal ingestion',
 67: 'More-than-usual meal ingestion',
 68: 'Less-than-usual meal ingestion',
 69: 'Typical exercise activity',
 70: 'More-than-usual exercise activity',
 71: 'Less-than-usual exercise activity',
 72: 'Unspecified special event'}

In [28]:
glucose_indices = list(range(57,65))
#patient_1['Glucose measurement'] = patient_1.apply(lambda x: x[glucose_indices].mean(), axis=1)

In [29]:
patient_1.rename(columns=code_dict)

2,Regular insulin dose,NPH insulin dose,Unspecified blood glucose measurement,Pre-breakfast blood glucose measurement,Pre-lunch blood glucose measurement,Pre-supper blood glucose measurement,Hypoglycemic symptoms
timestamp,,,,,,,
1991-04-21 09:09:00,9.0,13.0,NaN,100.0,NaN,NaN,NaN
1991-04-21 17:08:00,7.0,NaN,NaN,NaN,NaN,119.0,NaN
1991-04-21 22:51:00,NaN,NaN,123.0,NaN,NaN,NaN,NaN
1991-04-22 07:35:00,10.0,13.0,NaN,216.0,NaN,NaN,NaN
1991-04-22 13:40:00,2.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1991-09-02 08:51:00,9.0,16.0,NaN,168.0,NaN,NaN,NaN
1991-09-02 13:00:00,4.0,NaN,NaN,NaN,NaN,NaN,NaN
1991-09-02 17:30:00,7.0,NaN,NaN,NaN,NaN,61.0,NaN


In [30]:
patient_1 =  df[df['patient_id']==1]
patient_1 = patient_1.pivot(index='timestamp', columns=2, values=3)

In [31]:
df.isna().any()

0             False
1             False
2             False
3             False
patient_id    False
timestamp     False
dtype: bool

In [32]:
df

,0,1,2,3,patient_id,timestamp
0,04-21-1991,9:09,58,100,1,1991-04-21 09:09:00
1,04-21-1991,9:09,33,9,1,1991-04-21 09:09:00
2,04-21-1991,9:09,34,13,1,1991-04-21 09:09:00
3,04-21-1991,17:08,62,119,1,1991-04-21 17:08:00
4,04-21-1991,17:08,33,7,1,1991-04-21 17:08:00
...,...,...,...,...,...,...
29325,05-09-1989,08:00,33,1,70,1989-05-09 08:00:00
29326,05-09-1989,08:00,34,7,70,1989-05-09 08:00:00
29327,05-10-1989,08:00,34,7,70,1989-05-10 08:00:00
29328,05-11-1989,08:00,34,7,70,1989-05-11 08:00:00


In [33]:
tidy1 = df.pivot_table(index=['timestamp', 'patient_id'], columns=2, values=3).sort_values(['patient_id', 'timestamp']).reset_index().copy()

In [34]:
def display_tablenames(df):
    return df.rename(columns=code_dict)

In [35]:
display_tablenames(tidy1)

2,timestamp,patient_id,Regular insulin dose,NPH insulin dose,UltraLente insulin dose,Unspecified blood glucose measurement,Pre-breakfast blood glucose measurement,Post-breakfast blood glucose measurement,Pre-lunch blood glucose measurement,Post-lunch blood glucose measurement,...,Post-supper blood glucose measurement,Pre-snack blood glucose measurement,Hypoglycemic symptoms,Typical meal ingestion,More-than-usual meal ingestion,Less-than-usual meal ingestion,Typical exercise activity,More-than-usual exercise activity,Less-than-usual exercise activity,Unspecified special event
0,1991-04-21 09:09:00,1,9.0,13.0,NaN,NaN,100.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1991-04-21 17:08:00,1,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1991-04-21 22:51:00,1,NaN,NaN,NaN,123.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1991-04-22 07:35:00,1,10.0,13.0,NaN,NaN,216.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1991-04-22 13:40:00,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18198,1989-05-08 22:00:00,70,NaN,NaN,NaN,145.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18199,1989-05-09 08:00:00,70,1.0,7.0,NaN,NaN,259.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18200,1989-05-10 08:00:00,70,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18201,1989-05-11 08:00:00,70,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
tidy1['measurement_type'] = tidy1[glucose_indices].apply(lambda x: code_dict.get(x.idxmax(), np.nan), axis=1)

In [37]:
tidy1['glucose measurement'] = tidy1[glucose_indices].apply(lambda x: x.max(), axis=1)

In [38]:
tidy1

2,timestamp,patient_id,33,34,35,57,58,59,60,61,...,65,66,67,68,69,70,71,72,measurement_type,glucose measurement
0,1991-04-21 09:09:00,1,9.0,13.0,NaN,NaN,100.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pre-breakfast blood glucose measurement,100.0
1,1991-04-21 17:08:00,1,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pre-supper blood glucose measurement,119.0
2,1991-04-21 22:51:00,1,NaN,NaN,NaN,123.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unspecified blood glucose measurement,123.0
3,1991-04-22 07:35:00,1,10.0,13.0,NaN,NaN,216.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pre-breakfast blood glucose measurement,216.0
4,1991-04-22 13:40:00,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18198,1989-05-08 22:00:00,70,NaN,NaN,NaN,145.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unspecified blood glucose measurement,145.0
18199,1989-05-09 08:00:00,70,1.0,7.0,NaN,NaN,259.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pre-breakfast blood glucose measurement,259.0
18200,1989-05-10 08:00:00,70,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18201,1989-05-11 08:00:00,70,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# tidy1[tidy1[glucose_indices].apply(lambda x: x.max(), axis=1)!=tidy1[glucose_indices].apply(lambda x: x.mean(), axis=1)][tidy1[tidy1[glucose_indices].apply(lambda x: x.max(), axis=1)!=tidy1[glucose_indices].apply(lambda x: x.mean(), axis=1)]['measurement_type']=='Unspecified blood glucose measurement'].iloc[0]

In [40]:
# Meal Ingestions
print('Rows with meal ingestions:', tidy1[[66,67,68]].notna().any(axis=1).sum())
# Hypoglycemic symptoms 
print('Hypoglycemic symptoms :', tidy1[[65]].notna().any(axis=1).sum())
# Exercise activity
print('Exercise activity :', tidy1[[69, 70, 71]].notna().any(axis=1).sum())
# Exercise activity
print('Rows with Unspecified event:',tidy1[[72]].notna().any(axis=1).sum())
#total rows to drop:
print('Total rows affected: ', tidy1[[65,66,67,68,69,70,71,72]].notna().any(axis=1).sum())

Rows with meal ingestions: 512
Hypoglycemic symptoms : 328
Exercise activity : 305
Rows with Unspecified event: 94
Total rows affected:  1150


In [41]:
tidy1['glucose measurement'].notna().sum()

13517

In [42]:
tidy1[tidy1['glucose measurement'].notna()][[65,66,67,68,69,70,71,72]].notna().any(axis=1).sum()

120

In [43]:
patient_1.rename(columns=code_dict)

2,Regular insulin dose,NPH insulin dose,Unspecified blood glucose measurement,Pre-breakfast blood glucose measurement,Pre-lunch blood glucose measurement,Pre-supper blood glucose measurement,Hypoglycemic symptoms
timestamp,,,,,,,
1991-04-21 09:09:00,9.0,13.0,NaN,100.0,NaN,NaN,NaN
1991-04-21 17:08:00,7.0,NaN,NaN,NaN,NaN,119.0,NaN
1991-04-21 22:51:00,NaN,NaN,123.0,NaN,NaN,NaN,NaN
1991-04-22 07:35:00,10.0,13.0,NaN,216.0,NaN,NaN,NaN
1991-04-22 13:40:00,2.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1991-09-02 08:51:00,9.0,16.0,NaN,168.0,NaN,NaN,NaN
1991-09-02 13:00:00,4.0,NaN,NaN,NaN,NaN,NaN,NaN
1991-09-02 17:30:00,7.0,NaN,NaN,NaN,NaN,61.0,NaN


In [44]:
# Usable data
usable_df = tidy1[tidy1['glucose measurement'].notna()]
usable_df

2,timestamp,patient_id,33,34,35,57,58,59,60,61,...,65,66,67,68,69,70,71,72,measurement_type,glucose measurement
0,1991-04-21 09:09:00,1,9.0,13.0,NaN,NaN,100.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pre-breakfast blood glucose measurement,100.0
1,1991-04-21 17:08:00,1,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pre-supper blood glucose measurement,119.0
2,1991-04-21 22:51:00,1,NaN,NaN,NaN,123.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unspecified blood glucose measurement,123.0
3,1991-04-22 07:35:00,1,10.0,13.0,NaN,NaN,216.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pre-breakfast blood glucose measurement,216.0
5,1991-04-22 16:56:00,1,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pre-supper blood glucose measurement,211.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18193,1989-05-07 12:00:00,70,NaN,NaN,NaN,NaN,NaN,NaN,151.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pre-lunch blood glucose measurement,151.0
18195,1989-05-07 22:00:00,70,NaN,NaN,NaN,265.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unspecified blood glucose measurement,265.0
18196,1989-05-08 08:00:00,70,1.0,7.0,NaN,NaN,248.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pre-breakfast blood glucose measurement,248.0
18198,1989-05-08 22:00:00,70,NaN,NaN,NaN,145.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unspecified blood glucose measurement,145.0


In [45]:
# split dataset to train and test PATIENTS
# our model should generalize to patients outside of the train data
test_patients = np.random.choice(usable_df['patient_id'].unique(),10)

In [46]:
test_df = usable_df[usable_df['patient_id'].isin(test_patients)]
train_df = usable_df[~usable_df['patient_id'].isin(test_patients)]

In [47]:
# Baseline prediction to beat
baseline_pred = train_df['glucose measurement'].mean()

In [48]:
# Split test patients to known data and test data
# This form assumes that we would have a certain amount of known data to help our model
known_data = []
test_data = []
for _id in test_df['patient_id'].unique():
    known, test = np.array_split(test_df[test_df['patient_id']==_id], 2)
    known_data.append(known)
    test_data.append(test)
    
real_test_df = pd.concat(test_data)
known_df = pd.concat(known_data)

In [49]:
y_true = real_test_df['glucose measurement']
y_pred = np.array(y_true.size * [baseline_pred])

In [50]:
# Gather preliminary metrics
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae = mean_absolute_error(y_true, y_pred)
print(f'RMSE: {rmse}\nMAE {mae}')

RMSE: 80.24527709949714
MAE 63.70904339036541


In [51]:
# Because our dataset consists of individual patient data and some patients have more observation than the others
# It's ideal to for our metrics to average out our metrics to counter the imbalance and to
# prevent our metrics from overfitting to certain patient types
rmse_errors = []
mae_errors = []
for pid in real_test_df['patient_id'].unique():
    y_true = real_test_df[real_test_df['patient_id']==pid]['glucose measurement']
    y_pred = np.array(y_true.size * [baseline_pred])
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    rmse_errors.append(rmse)
    mae_errors.append(mae)
print(f'MRMSE: {np.mean(rmse_errors)}\nMMAE {np.mean(mae_errors)}')

MRMSE: 84.53171547517789
MMAE 69.68518291103695
